In [34]:
import keras as ks
import tensorflow as tf
import numpy as np
import cv2 as cv
import os
from PIL import Image
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator

In [35]:
def create_image_data(root):
    images = []
    for subdir, dirs, files in os.walk(root):
        for file in files:
            read_file = subdir + "/" + file
            img = cv.imread(read_file,cv.IMREAD_GRAYSCALE)
            if ('ipynb' not in read_file and 'DS' not in read_file):
                images.append(img)
    return np.array(images)

## Data Preprossesing

This will generate the image data and normalize the data to be between 0 and 1.
Darker pixels will be closer to 0 and lighter pixels will be closer to 0. Furthermore, we are using grey-scale .

In [36]:
frown = create_image_data("data/60frowns/")/255
smile = create_image_data("data/60smiles/")/255

frowns = frown.reshape(frown.shape[0],60,60,1).astype('float32')
smiles = smile.reshape(smile.shape[0],60,60,1).astype('float32')

X = np.concatenate([frowns, smiles])

In [37]:
frowns_y = np.repeat(0, frown.shape[0])
smiles_y = np.repeat(1, smile.shape[0])

y = np.concatenate([frowns_y,smiles_y])

Splits the data intro training and testing sets.

In [38]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Using the ImageDataGenerator we will create additional images for the model to trail on since we only have 360 images.

In [39]:
img_gen = ImageDataGenerator(rotation_range=20)
img_gen.fit(X_train)

## CNN Model
This will train and build the CNN model. 

### Layers
    1. Convolutional layer: 
        - Input shape: (60, 60, 1)
        - RelU activation function a' = max(0, a) for each pixel
        - L2 regularization with a = 0.001, multiply each weight in training by 0.001
    2. Pooling layer:
        - Pooling size = (3,3)
    3. Dense layer (Fully Connected layer):
        - Output size of 1
        - Sigmoid activation function
### Compiling the model
    - We will use a binary_crossentropy loss function
    - RMsprop optimizer function
### Fitting the model
#### ImageDataGenerator
    - A key detail about our model is that we will be generating 30 new images each epoch during the training of the model.

In [40]:
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras import regularizers
from keras import optimizers

In [44]:
model = ks.Sequential()

model.add(Conv2D(filters=15,kernel_size=(6,6),activation='relu', input_shape=(60,60,1), kernel_regularizer=regularizers.l2(0.001)))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(units=1,activation='sigmoid'))

model.compile(loss='binary_crossentropy',optimizer='RMSprop', metrics = ['accuracy'])
history_model = model.fit_generator(img_gen.flow(X_train,y_train, batch_size=30),steps_per_epoch=45, epochs=10);
history_evaluate = model.evaluate(X_test, y_test);
history_evaluate

Epoch 1/10
45/45 [==============================] - 2s 39ms/step - loss: 0.5045 - acc: 0.8045
Epoch 2/10
45/45 [==============================] - 2s 37ms/step - loss: 0.2032 - acc: 0.9407
Epoch 3/10
45/45 [==============================] - 2s 34ms/step - loss: 0.0858 - acc: 0.9815
Epoch 4/10
45/45 [==============================] - 2s 34ms/step - loss: 0.0435 - acc: 0.9889
Epoch 5/10
45/45 [==============================] - 2s 34ms/step - loss: 0.0264 - acc: 0.9948
Epoch 6/10
45/45 [==============================] - 2s 35ms/step - loss: 0.0154 - acc: 0.9970
Epoch 7/10
45/45 [==============================] - 2s 34ms/step - loss: 0.0103 - acc: 1.0000
Epoch 8/10
45/45 [==============================] - 2s 34ms/step - loss: 0.0092 - acc: 0.9993
Epoch 9/10
45/45 [==============================] - 2s 34ms/step - loss: 0.0070 - acc: 1.0000
Epoch 10/10
108/108 [==============================] - 0s 1ms/step


[0.05919806833413464, 0.9814814814814815]

In [43]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 55, 55, 15)        555       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 27, 27, 15)        0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 10935)             0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 10935)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 10936     
Total params: 11,491
Trainable params: 11,491
Non-trainable params: 0
_________________________________________________________________


## Interesting observations to consider and evaluate

In [45]:
frown = create_image_data("badfrowns/")/255
smile = create_image_data("badsmiles/")/255

frowns = frown.reshape(frown.shape[0],60,60,1).astype('float32')
smiles = smile.reshape(smile.shape[0],60,60,1).astype('float32')

X = np.concatenate([frowns, smiles])

frowns_y = np.repeat(0, frown.shape[0])
smiles_y = np.repeat(1, smile.shape[0])

y = np.concatenate([frowns_y,smiles_y])

In [46]:
history_evaluate = model.evaluate(X, y);
history_evaluate

36/36 [==============================] - 0s 536us/step


[0.31725090079837376, 0.9444444444444444]

What about Dekhtyar?

In [25]:
dek_happy = create_image_data("dekhtyar/")
dek_sadface = create_image_data("dekh_sad/")/255

dek_happiness = dek_happy[0].reshape(1,60,60,1).astype('float32')
dek_sadfaces = dek_sadface[0].reshape(1,60,60,1).astype('float32')

X = np.concatenate([dek_happiness, dek_sadfaces])

deks_1_y = np.repeat(0, dek_happiness.shape[0])
deks_2_y = np.repeat(1, dek_sadfaces.shape[0])

y = np.concatenate([deks_1_y,deks_2_y])

First picture fails

In [26]:
history_evaluate = model.evaluate(dek_happiness,deks_1_y);
history_evaluate

1/1 [==============================] - 0s 2ms/step


[0.0030422767158597708, 1.0]

Second picture fails

In [27]:
dek_happiness = dek_happy[1].reshape(1,60,60,1).astype('float32')
deks_1_y = np.repeat(0, dek_happiness.shape[0])

In [28]:
history_evaluate = model.evaluate(dek_happiness,deks_1_y);
history_evaluate

1/1 [==============================] - 0s 2ms/step


[0.0030422767158597708, 1.0]

Third picture success

In [29]:
dek_happiness = dek_happy[2].reshape(1,60,60,1).astype('float32')
deks_1_y = np.repeat(0, dek_happiness.shape[0])

In [30]:
history_evaluate = model.evaluate(dek_happiness,deks_1_y);
history_evaluate

1/1 [==============================] - 0s 2ms/step


[0.0030422767158597708, 1.0]

Fourth picture success

In [31]:
dek_sadfaces = dek_sadface[0].reshape(1,60,60,1).astype('float32')
deks_1_y = np.repeat(0, dek_happiness.shape[0])

In [32]:
history_evaluate = model.evaluate(dek_sadfaces,deks_1_y);
history_evaluate

1/1 [==============================] - 0s 4ms/step


[0.49814361333847046, 1.0]